
#Previsão de Risco de Burnout no Ambiente de Trabalho

Este notebook tem como objetivo treinar e avaliar modelos de Machine Learning: KNN, Árvore de Classificação, Naive Bayes e SVM, para prever o risco de burnout entre colaboradores com base em variáveis como jornada de trabalho, suporte no ambiente, equilíbrio vida-trabalho, entre outras.

Projeto desenvolvido por Ágata Velasco Penze


In [1]:

# Importando bibliotecas
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
import joblib


In [2]:

# Carregando os dados via URL do GitHub
url = "https://raw.githubusercontent.com/agatavelasco/mvp-machine-learning/refs/heads/master/dataset/mental_health_workplace_survey.csv"
df = pd.read_csv(url)
df.head()


,EmployeeID,Age,Gender,Country,JobRole,Department,YearsAtCompany,WorkHoursPerWeek,RemoteWork,BurnoutLevel,...,CommuteTime,HasMentalHealthSupport,ManagerSupportScore,HasTherapyAccess,MentalHealthDaysOff,SalaryRange,WorkLifeBalanceScore,TeamSize,CareerGrowthScore,BurnoutRisk
0,1001,50,Male,UK,Sales Associate,HR,14,47,No,3.37,...,117,No,3.15,Yes,8,40K-60K,8.82,6,9.20,0
1,1002,36,Male,Germany,Software Engineer,IT,1,59,Hybrid,7.39,...,8,Yes,4.40,Yes,4,80K-100K,2.80,45,8.46,1
2,1003,29,Non-binary,India,IT Admin,IT,13,59,Hybrid,7.10,...,75,No,3.63,No,6,80K-100K,7.28,7,7.96,1
3,1004,42,Male,Australia,HR Specialist,IT,15,31,Yes,4.18,...,43,Yes,4.50,Yes,9,60K-80K,1.31,11,8.90,0
4,1005,40,Male,Brazil,Customer Support,Support,6,34,Yes,8.28,...,58,Yes,5.51,Yes,6,<40K,1.17,18,8.88,1


In [3]:

# Pré-processamento inicial
# Removendo colunas irrelevantes e codificando variáveis

df = df.drop(columns=["EmployeeID", "Country", "JobRole", "Department", "SalaryRange"])

# Convertendo colunas categóricas
label_cols = ["Gender", "RemoteWork", "HasMentalHealthSupport", "HasTherapyAccess"]
for col in label_cols:
    df[col] = LabelEncoder().fit_transform(df[col])

# Definindo X e y
X = df.drop(columns=["BurnoutRisk"])
y = df["BurnoutRisk"]

# Dividindo em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print("Formato do treino:", X_train.shape)


Formato do treino: (2400, 19)


In [4]:

# Treinando modelos com pipelines

models = {
    "KNN": KNeighborsClassifier(),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Naive Bayes": GaussianNB(),
    "SVM": SVC()
}

for name, model in models.items():
    pipeline = Pipeline([
        ("scaler", StandardScaler()),
        ("classifier", model)
    ])
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print(f"\nModelo: {name}")
    print(f"Acurácia: {acc:.2f}")
    print("Classification Report:")
    print(classification_report(y_test, y_pred))



Modelo: KNN
Acurácia: 0.84
Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.92      0.89       404
           1       0.81      0.68      0.74       196

    accuracy                           0.84       600
   macro avg       0.83      0.80      0.81       600
weighted avg       0.84      0.84      0.84       600


Modelo: Decision Tree
Acurácia: 1.00
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       404
           1       1.00      1.00      1.00       196

    accuracy                           1.00       600
   macro avg       1.00      1.00      1.00       600
weighted avg       1.00      1.00      1.00       600


Modelo: Naive Bayes
Acurácia: 0.99
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       404
           1       0.99      0.99      0.99       196

    accurac

In [7]:

# Exportando o melhor modelo (ex: SVM)

final_model = Pipeline([
    ("scaler", StandardScaler()),
    ("classifier", SVC())
])
final_model.fit(X_train, y_train)

joblib.dump(final_model, "burnout_model.pkl")
print("Modelo salvo como 'burnout_model.pkl'")
from google.colab import files
files.download("burnout_model.pkl")


Modelo salvo como 'burnout_model.pkl'


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


## ✅ Conclusão

O modelo treinado apresenta desempenho satisfatório para prever o risco de burnout com base nas variáveis analisadas. Esta aplicação pode ser estendida para sistemas de RH ou saúde corporativa com foco em prevenção e bem-estar do colaborador.

Na próxima etapa, este modelo será integrado a uma aplicação full stack que permitirá a inserção de novos dados e previsão do risco em tempo real.
